In [1]:
# VSCode中安装TA-Lib完整流程
# 在Jupyter Notebook中运行以下代码

# ==========================================
# 步骤1: 检查当前环境
# ==========================================

import sys
import platform
import subprocess
import os

print("🔍 当前环境检查")
print("=" * 50)
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"架构: {platform.machine()}")
print(f"Python版本: {sys.version}")
print(f"Python路径: {sys.executable}")
print(f"当前工作目录: {os.getcwd()}")

# ==========================================
# 步骤2: 安装必要的工具和依赖
# ==========================================

def run_command(command, description=""):
    """运行shell命令并显示结果"""
    print(f"\n🚀 {description}")
    print(f"执行命令: {command}")
    print("-" * 40)
    
    try:
        # 使用subprocess运行命令
        result = subprocess.run(command, 
                               shell=True, 
                               capture_output=True, 
                               text=True, 
                               timeout=300)  # 5分钟超时
        
        if result.stdout:
            print("✅ 输出:")
            print(result.stdout)
        
        if result.stderr and result.returncode != 0:
            print("⚠️ 错误信息:")
            print(result.stderr)
            return False
        
        if result.returncode == 0:
            print(f"✅ {description} 执行成功!")
            return True
        else:
            print(f"❌ {description} 执行失败!")
            return False
            
    except subprocess.TimeoutExpired:
        print("⏰ 命令执行超时")
        return False
    except Exception as e:
        print(f"❌ 执行出错: {e}")
        return False

# 步骤2.1: 检查并安装Xcode命令行工具
print("\n📱 检查Xcode命令行工具...")
xcode_result = run_command("xcode-select --install", "安装Xcode命令行工具")
if not xcode_result:
    print("ℹ️ Xcode工具可能已安装或需要手动安装")

# 步骤2.2: 检查Homebrew
print("\n🍺 检查Homebrew...")
brew_check = run_command("brew --version", "检查Homebrew版本")

if not brew_check:
    print("🔧 Homebrew未安装，开始安装...")
    brew_install_cmd = '/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"'
    brew_result = run_command(brew_install_cmd, "安装Homebrew")
    
    if brew_result:
        # 添加Homebrew到PATH
        print("🔧 配置Homebrew环境变量...")
        path_commands = [
            'echo \'export PATH="/opt/homebrew/bin:$PATH"\' >> ~/.zshrc',
            'export PATH="/opt/homebrew/bin:$PATH"'
        ]
        for cmd in path_commands:
            run_command(cmd, f"执行: {cmd}")
else:
    print("✅ Homebrew已安装")

# ==========================================
# 步骤3: 安装TA-Lib C库
# ==========================================

print("\n📊 安装TA-Lib C库...")
talib_c_result = run_command("brew install ta-lib", "安装TA-Lib C库")

if talib_c_result:
    print("✅ TA-Lib C库安装成功!")
else:
    print("⚠️ TA-Lib C库安装可能有问题，继续尝试...")

# ==========================================
# 步骤4: 设置编译环境变量
# ==========================================

print("\n🔧 设置编译环境变量...")

# 为M1/M2 Mac设置正确的路径
if platform.machine() == 'arm64':
    homebrew_prefix = '/opt/homebrew'
else:
    homebrew_prefix = '/usr/local'

os.environ['CPPFLAGS'] = f'-I{homebrew_prefix}/include'
os.environ['LDFLAGS'] = f'-L{homebrew_prefix}/lib'

print(f"CPPFLAGS: {os.environ.get('CPPFLAGS')}")
print(f"LDFLAGS: {os.environ.get('LDFLAGS')}")

# ==========================================
# 步骤5: 安装Python TA-Lib包
# ==========================================

print("\n🐍 安装Python TA-Lib包...")

# 先升级pip
pip_upgrade = run_command(f"{sys.executable} -m pip install --upgrade pip", "升级pip")

# 安装TA-Lib
install_commands = [
    f"{sys.executable} -m pip install --no-cache-dir TA-Lib",
    f"{sys.executable} -m pip install --user --no-cache-dir TA-Lib",
    f"CPPFLAGS=-I{homebrew_prefix}/include LDFLAGS=-L{homebrew_prefix}/lib {sys.executable} -m pip install --no-cache-dir TA-Lib"
]

talib_installed = False
for i, cmd in enumerate(install_commands, 1):
    print(f"\n📦 尝试安装方法 {i}...")
    if run_command(cmd, f"TA-Lib安装方法{i}"):
        talib_installed = True
        break
    else:
        print(f"⚠️ 方法{i}失败，尝试下一个方法...")

# ==========================================
# 步骤6: 验证安装
# ==========================================

print("\n🧪 验证TA-Lib安装...")

try:
    import talib
    import numpy as np
    
    print("✅ TA-Lib导入成功!")
    print(f"📦 TA-Lib版本: {talib.__version__}")
    print(f"🔧 可用函数数量: {len(talib.get_functions())}")
    
    # 测试基本功能
    test_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=float)
    sma_result = talib.SMA(test_data, timeperiod=5)
    print(f"🎯 SMA测试结果: {sma_result[-1]:.2f}")
    
    # 显示可用的部分函数
    functions = talib.get_functions()
    print(f"\n📈 部分可用技术指标:")
    for func in functions[:10]:
        print(f"   - {func}")
    print(f"   ... 还有{len(functions)-10}个函数")
    
    print("\n🎉 TA-Lib安装和测试完全成功!")
    
except ImportError as e:
    print(f"❌ TA-Lib导入失败: {e}")
    print("\n🔧 可能的解决方案:")
    print("1. 重启Jupyter Kernel")
    print("2. 重启VSCode")
    print("3. 检查Python环境是否正确")
    print("4. 使用替代方案 (ta 或 pandas_ta)")
    
except Exception as e:
    print(f"⚠️ TA-Lib测试失败: {e}")

# ==========================================
# 步骤7: 替代方案（如果TA-Lib安装失败）
# ==========================================

print("\n💡 安装替代技术指标库 (备用方案)...")

# 安装替代库
alternative_libs = [
    ("ta", "纯Python技术分析库"),
    ("pandas_ta", "pandas技术分析扩展"),
    ("yfinance", "Yahoo财经数据获取")
]

for lib_name, description in alternative_libs:
    cmd = f"{sys.executable} -m pip install {lib_name}"
    if run_command(cmd, f"安装{lib_name} - {description}"):
        try:
            __import__(lib_name)
            print(f"✅ {lib_name} 安装并验证成功")
        except:
            print(f"⚠️ {lib_name} 安装成功但导入失败")

print("\n" + "="*50)
print("🏁 安装流程完成!")
print("="*50)
print("\n📋 后续步骤:")
print("1. 如果TA-Lib安装成功，您可以直接使用")
print("2. 如果失败，请重启Jupyter Kernel后重试")
print("3. 或者使用替代库 (ta, pandas_ta)")
print("4. 运行下面的测试代码验证功能")

🔍 当前环境检查
操作系统: Darwin 24.6.0
架构: arm64
Python版本: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Python路径: /opt/anaconda3/bin/python
当前工作目录: /

📱 检查Xcode命令行工具...

🚀 安装Xcode命令行工具
执行命令: xcode-select --install
----------------------------------------
⚠️ 错误信息:
xcode-select: note: Command line tools are already installed. Use "Software Update" in System Settings or the softwareupdate command line interface to install updates

ℹ️ Xcode工具可能已安装或需要手动安装

🍺 检查Homebrew...

🚀 检查Homebrew版本
执行命令: brew --version
----------------------------------------
⚠️ 错误信息:
/bin/sh: brew: command not found

🔧 Homebrew未安装，开始安装...

🚀 安装Homebrew
执行命令: /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
----------------------------------------
✅ 输出:
==> Checking for `sudo` access (which may request your password)...

⚠️ 错误信息:
Need sudo access on macOS (e.g. the user jackstudio needs to be an Administrator)!


📊 安装TA-Lib C库...

🚀 安装TA-Lib C

In [1]:
# TA-Lib安装成功！现在验证功能
print("🎉 TA-Lib安装成功验证")
print("=" * 50)

try:
    import talib
    import numpy as np
    import pandas as pd
    
    print("✅ TA-Lib导入成功!")
    print(f"📦 TA-Lib版本: {talib.__version__}")
    print(f"🔧 可用函数数量: {len(talib.get_functions())}")
    
    # 测试基本功能
    print("\n🧪 基本功能测试...")
    test_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=float)
    sma_result = talib.SMA(test_data, timeperiod=5)
    print(f"✅ SMA测试结果: {sma_result[-1]:.2f}")
    
    # 创建更真实的测试数据
    np.random.seed(42)
    n_points = 100
    close_data = 100 + np.cumsum(np.random.randn(n_points) * 0.5)
    high_data = close_data + np.random.rand(n_points) * 2
    low_data = close_data - np.random.rand(n_points) * 2
    open_data = close_data + (np.random.rand(n_points) - 0.5) * 1
    volume_data = np.random.randint(1000, 5000, n_points).astype(float)
    
    print(f"\n📈 使用真实数据测试 ({n_points}个数据点)...")
    
    # 测试各种技术指标
    indicators_results = {}
    
    # 移动平均
    indicators_results['SMA_20'] = talib.SMA(close_data, timeperiod=20)
    indicators_results['EMA_12'] = talib.EMA(close_data, timeperiod=12)
    print(f"✅ 移动平均: SMA(20)={indicators_results['SMA_20'][-1]:.2f}, EMA(12)={indicators_results['EMA_12'][-1]:.2f}")
    
    # 动量指标
    indicators_results['RSI'] = talib.RSI(close_data, timeperiod=14)
    macd, signal, hist = talib.MACD(close_data)
    indicators_results['MACD'] = macd
    print(f"✅ 动量指标: RSI={indicators_results['RSI'][-1]:.2f}, MACD={macd[-1]:.4f}")
    
    # 波动性指标
    bb_upper, bb_middle, bb_lower = talib.BBANDS(close_data)
    atr = talib.ATR(high_data, low_data, close_data, timeperiod=14)
    print(f"✅ 波动性指标: 布林带中轨={bb_middle[-1]:.2f}, ATR={atr[-1]:.2f}")
    
    # 趋势指标
    adx = talib.ADX(high_data, low_data, close_data, timeperiod=14)
    print(f"✅ 趋势指标: ADX={adx[-1]:.2f}")
    
    # K线形态识别
    doji = talib.CDLDOJI(open_data, high_data, low_data, close_data)
    hammer = talib.CDLHAMMER(open_data, high_data, low_data, close_data)
    doji_count = np.sum(doji != 0)
    hammer_count = np.sum(hammer != 0)
    print(f"✅ K线形态: 十字星={doji_count}个, 锤子线={hammer_count}个")
    
    # 成交量指标
    obv = talib.OBV(close_data, volume_data)
    print(f"✅ 成交量指标: OBV={obv[-1]:.0f}")
    
    print(f"\n📊 显示部分可用技术指标函数:")
    functions = talib.get_functions()
    
    # 按类别显示函数
    function_groups = {
        '重叠研究': ['SMA', 'EMA', 'WMA', 'DEMA', 'TEMA', 'TRIMA', 'KAMA', 'MAMA', 'T3'],
        '动量指标': ['RSI', 'MACD', 'STOCH', 'STOCHF', 'WILLR', 'CCI', 'CMO', 'ROC'],
        '成交量指标': ['OBV', 'AD', 'ADOSC', 'MFI'],
        '波动性指标': ['ATR', 'NATR', 'TRANGE'],
        '价格变换': ['AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE'],
        '周期指标': ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE'],
        '数学变换': ['ADD', 'DIV', 'MAX', 'MIN', 'SUM', 'LN', 'LOG10', 'SQRT'],
        '数学运算符': ['ACOS', 'ASIN', 'ATAN', 'COS', 'SIN', 'TAN', 'SINH', 'COSH', 'TANH'],
        '统计函数': ['BETA', 'CORREL', 'LINEARREG', 'STDDEV', 'TSF', 'VAR']
    }
    
    for group_name, group_functions in function_groups.items():
        available_in_group = [f for f in group_functions if f in functions]
        if available_in_group:
            print(f"   {group_name}: {', '.join(available_in_group[:5])}" + 
                  (f" ...等{len(available_in_group)}个" if len(available_in_group) > 5 else ""))
    
    print(f"\n🎯 量化交易框架兼容性测试...")
    
    # 创建pandas DataFrame测试
    df = pd.DataFrame({
        'open': open_data,
        'high': high_data,
        'low': low_data,
        'close': close_data,
        'volume': volume_data
    })
    
    # 添加技术指标到DataFrame
    df['SMA_20'] = talib.SMA(df['close'].values, timeperiod=20)
    df['RSI_14'] = talib.RSI(df['close'].values, timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'].values)
    
    print(f"✅ pandas DataFrame集成成功")
    print(f"   数据形状: {df.shape}")
    print(f"   特征列数: {df.select_dtypes(include=[np.number]).shape[1]}")
    
    print(f"\n🎉 TA-Lib完全可用于您的量化交易框架！")
    print(f"✅ 所有核心功能测试通过")
    print(f"✅ 与pandas完美兼容")
    print(f"✅ 支持60+技术指标")
    print(f"✅ 支持K线形态识别")
    
    # 显示最新数据示例
    print(f"\n📋 最新5个数据点示例:")
    print(df[['close', 'SMA_20', 'RSI_14', 'MACD']].tail().round(2))
    
except ImportError as e:
    print(f"❌ 导入失败: {e}")
except Exception as e:
    print(f"❌ 测试失败: {e}")

print(f"\n🚀 现在您可以更新特征工程模块了！")
print(f"💡 将原来的 import talib 错误处理代码替换为直接导入")

🎉 TA-Lib安装成功验证
✅ TA-Lib导入成功!
📦 TA-Lib版本: 0.6.5
🔧 可用函数数量: 158

🧪 基本功能测试...
✅ SMA测试结果: 8.00

📈 使用真实数据测试 (100个数据点)...
✅ 移动平均: SMA(20)=95.30, EMA(12)=95.15
✅ 动量指标: RSI=43.20, MACD=-0.1657
✅ 波动性指标: 布林带中轨=94.82, ATR=2.10
✅ 趋势指标: ADX=14.85
✅ K线形态: 十字星=39个, 锤子线=1个
✅ 成交量指标: OBV=-10395

📊 显示部分可用技术指标函数:
   重叠研究: SMA, EMA, WMA, DEMA, TEMA ...等9个
   动量指标: RSI, MACD, STOCH, STOCHF, WILLR ...等8个
   成交量指标: OBV, AD, ADOSC, MFI
   波动性指标: ATR, NATR, TRANGE
   价格变换: AVGPRICE, MEDPRICE, TYPPRICE, WCLPRICE
   周期指标: HT_DCPERIOD, HT_DCPHASE, HT_PHASOR, HT_SINE, HT_TRENDMODE
   数学变换: ADD, DIV, MAX, MIN, SUM ...等8个
   数学运算符: ACOS, ASIN, ATAN, COS, SIN ...等9个
   统计函数: BETA, CORREL, LINEARREG, STDDEV, TSF ...等6个

🎯 量化交易框架兼容性测试...
✅ pandas DataFrame集成成功
   数据形状: (100, 10)
   特征列数: 10

🎉 TA-Lib完全可用于您的量化交易框架！
✅ 所有核心功能测试通过
✅ 与pandas完美兼容
✅ 支持60+技术指标
✅ 支持K线形态识别

📋 最新5个数据点示例:
    close  SMA_20  RSI_14  MACD
95  94.64   95.48   40.97 -0.06
96  94.79   95.42   42.82 -0.10
97  94.92   95.36   44.46 -0.13
98  94.92   95.31 